# 1. 分层索引(`MultiIndex`)
索引分层是pandas的重要特性，允许你在一个轴向上拥有多个（两个或两个以上）索引层次。笼统地说，分层索引提供了一种在更低纬度的形式中处理更高维度数据的方式。

注：索引是Pandas中的一个十分重要的对象类型，很多DataFrame或Series之间运算、修改、拼接都是依照索引来进行操作的。在Pandas的学习中，索引是一个重要，又很容易被忽略的点。

In [2]:
import pandas as pd
import numpy as np

In [3]:
# 以MultiIndex作为索引的Seires
data = pd.Series(np.random.randn(9), 
                 index=[list('aaabbccdd'),  # level 0
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])  # level 1
data

a  1    0.362862
   2   -0.629980
   3    0.460300
b  1   -0.713460
   3   -1.719562
c  1    0.515811
   2   -0.263820
d  2    0.613620
   3   -0.439084
dtype: float64

In [4]:
data.index  # 索引(n)

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
# 通过分层索引对象，也可以称为部分索引(v)，允许简洁地选出数据的子集
data['b']

1   -0.713460
3   -1.719562
dtype: float64

In [6]:
data[1]  # 输入整数，则是按照位置标签进行索引

-0.6299797227772944

In [7]:
data['b':'d']  # 直接通过level 0的索引标签进行切片

b  1   -0.713460
   3   -1.719562
c  1    0.515811
   2   -0.263820
d  2    0.613620
   3   -0.439084
dtype: float64

In [8]:
data[('a', 2):('c', 1)]  # 也可以通过MultiIndex进行切片，注意要以元组的形式传入

a  2   -0.629980
   3    0.460300
b  1   -0.713460
   3   -1.719562
c  1    0.515811
dtype: float64

In [9]:
data.loc[['b', 'd']]  # 可以以列表的形式进行索引，即将列表中索引所对应的值找出

b  1   -0.713460
   3   -1.719562
d  2    0.613620
   3   -0.439084
dtype: float64

In [10]:
# 在内部层级中进行选择
data.loc[:, 2]  # data.loc[level 0, level 1]

a   -0.62998
c   -0.26382
d    0.61362
dtype: float64

In [11]:
data

a  1    0.362862
   2   -0.629980
   3    0.460300
b  1   -0.713460
   3   -1.719562
c  1    0.515811
   2   -0.263820
d  2    0.613620
   3   -0.439084
dtype: float64

In [12]:
# unstack方法将数据在DataFrame中重新排列
# 即要将其中一层的行索引变成列索引。
data.unstack()  # 数据从”花括号结构“变成”表格结构“

,1,2,3
a,0.362862,-0.62998,0.460300
b,-0.713460,NaN,-1.719562
c,0.515811,-0.26382,NaN
d,NaN,0.61362,-0.439084


In [13]:
# 如果是多层索引，则以上函数是针对内层索引（这里是store）。利用level可以选择具体哪层索引。
data.unstack(level=0)  

,a,b,c,d
1,0.362862,-0.713460,0.515811,NaN
2,-0.629980,NaN,-0.263820,0.613620
3,0.460300,-1.719562,NaN,-0.439084


In [14]:
# stack是unstack的反向操作
data.unstack().stack()  

a  1    0.362862
   2   -0.629980
   3    0.460300
b  1   -0.713460
   3   -1.719562
c  1    0.515811
   2   -0.263820
d  2    0.613620
   3   -0.439084
dtype: float64

In [15]:
# DataFrame中，每个轴都可以拥有分层索引
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                     columns=[['Ohio', 'Ohio', 'Colorade'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorade
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
# 分层的层级可以有名称
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorade
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
# 也可以用MultiIndex构造
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorade'], ['Green', 'Red', 'Green']], 
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorade', 'Green')],
           names=['state', 'color'])

In [18]:
frame.loc['b', 'Ohio']

color,Green,Red
key2,,
1,6,7
2,9,10


## 1.1 重排序和层级排序
有时候需要重新排列轴上的层级顺序，或者按照特点层级的值对数据进行排序

In [23]:
# swaplevel索引层级互换，接受两个层级序号或层级名称
# 返回一个进行了层级变更的新对象
frame.swaplevel('state', 'color', axis=1)  

color     Green  Red    Green
state      Ohio Ohio Colorade
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

In [24]:
# sort_index只能在单一层级上对数据进行排序
# 可通过level参数指定排序的层级
frame.sort_index(level=1)  # 注意，层级依旧是从0开始

state      Ohio     Colorade
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [25]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorade
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 1.2 按层级进行汇总统计

In [27]:
frame.sum(level='key2')

state  Ohio     Colorade
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [29]:
frame.mean(level=1)

state  Ohio     Colorade
color Green Red    Green
key2                    
1         3   4        5
2         6   7        8

## 1.3 使用`DataFrame`的列进行索引

In [30]:
frame = pd.DataFrame({'a': range(7), 
                      'b':range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [32]:
# set_index设置列为索引
# 设置c列、d列为索引
frame2 = frame.set_index(['c', 'd'])  
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [34]:
# reset_index，是set_index的反向操作
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 2. 联合与合并数据集
- `pandas.merge`：根据一个或多个键按行进行连接。
- `pandas.concat`：使对象在轴向上进行粘合或堆叠。
- `combine_first`：实例方法允许将重叠的数据拼接在一起，以使用一个对象中的值填充另一个对象中的缺失值

## 2.1 数据库风格的`DataFrame`连接(`merge`)

In [51]:
df1 = pd.DataFrame({'key': list('bbacaab'), 
                    'data1': range(7)})
df2 = pd.DataFrame({'key': list('abd'), 
                    'data2': range(3)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [53]:
# pandas.merge(两个表格的拼接)
# 未指定连接的键信息
# merge会自动将重叠列名作为连接的键
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [29]:
# 尽管效果相同，但建议在实操过程中，通过参数指定连接键
# 这样才是好的实现
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [55]:
# 当每个对象的列名时不同时
# 可以分别指定连接的列名
df3 = pd.DataFrame({'lkey': list('bbacaab'), 
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': list('abd'), 
                    'data2': range(3)})
print(df3)
print(df4)
pd.merge(df3, df4, left_on='lkey', right_on='rkey')  # 返回两张表中键的交集

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [56]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


$$ how参数的不同连接类型 $$

| 选项 | 行为 |
| --- | --- |
| `inner`（默认） | 只对两张表都有的键的交集进行联合 |
| `left` | 对所有在左表的键进行联合 |
| `right` | 对所有右表的键进行联合 |
| `outer` | 对两张表都有的键的并集进行联合 |

In [57]:
df1 = pd.DataFrame({'key': list('bbacab'), 
                    'data1': range(6)})
df2 = pd.DataFrame({'key': list('ababd'), 
                    'data2': range(5)})
print(df1)
print(df2)
pd.merge(df1, df2, on='key', how='left')

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [58]:
pd.merge(df1, df2, on='key', how='right')

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,5.0,1
3,b,0.0,3
4,b,1.0,3
5,b,5.0,3
6,a,2.0,0
7,a,4.0,0
8,a,2.0,2
9,a,4.0,2


In [59]:
# 多个键进行合并时
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                     'key2': ['one', 'two', 'one'], 
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                      'key2': ['one', 'one', 'one', 'two'], 
                      'rval': [4, 5, 6, 7]})
print(left)
print(right)
pd.merge(left, right, on=['key1', 'key2'], how='outer')  # 多建看做一个元组数据来作为单个连接键使用

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [60]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [61]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 2.2 根据索引合并(`merge`)

In [62]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 
                      'value': range(6)})
right1 =pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [63]:
# 将列和索引作为连接键
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [64]:
# 多层索引数据
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
                      'key2': [2000, 2001, 2002, 2001 ,2002], 
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), 
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
                             [2001, 2000, 2000, 2000, 2001, 2002 ]],
                     columns=['event1', 'event2'])
print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [65]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [66]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'], 
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], 
                      index=['b', 'c', 'd', 'e'], 
                      columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [69]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True, indicator=True)

,Ohio,Nevada,Missouri,Alabama,_merge
a,1.0,2.0,NaN,NaN,left_only
b,NaN,NaN,7.0,8.0,right_only
c,3.0,4.0,9.0,10.0,both
d,NaN,NaN,11.0,12.0,right_only
e,5.0,6.0,13.0,14.0,both


In [68]:
pd.merge?

In [70]:
# join实例方法，用于按照索引合并
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [71]:
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [73]:
# join还支持在调用DataFrame的某一列上连接传递的DataFrame的索引
left1.join(right1, on='key')
# pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [74]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], 
                       index=list('acef'), columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [76]:
# 对于一些简单的索引-索引合并，可以像join方法传入一个DataFrame列表
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


In [55]:
left2.join([right2, another], how='outer')

/Users/mengyuanxu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7280: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(frames, axis=1, join=how, verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## 2.3 沿轴向连接(`concat`，`append`)


In [83]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [95]:
# append (row)
# 和list的append函数有些相似，都是在Series或DataFrame的尾部，按索引进行拼接
s1.append(s2)

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [84]:
# pd.concat(推荐)
# 当我们要拼接多个表格的时候，较为高效的方法便是将表格存放在一个列表里面，利用concat进行拼接
pd.concat([s1, s2, s3])  # 默认沿着axis=0的轴向生效，生成另一个Series

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [85]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [86]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [87]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [88]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [93]:
# keys参数实现连接轴向上创建一个多层索引
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [94]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [67]:
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'], axis=1)

/Users/mengyuanxu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [98]:
# =DataFrame
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], 
                   columns=['one', 'two'])
df2 = pd.DataFrame(5+np.arange(4).reshape(2, 2), index=['a', 'c'], 
                   columns=['three', 'four'])
print(df1)
print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [99]:
# append，行拼接
df1.append(df2)

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [103]:
# concat
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [104]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [109]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=list('abcd'))
df2 = pd.DataFrame(np.random.randn(2, 3), columns=list('bda'))
print(df1)
print(df2)
df = pd.concat([df1, df2], ignore_index=True)  # ignore_index=True产生一段新的索引
df

          a         b         c         d
0 -1.197042 -1.550646  0.122298 -0.214030
1 -1.133196  1.167596 -0.583023 -0.603532
2  0.882659  1.173026 -0.456927 -0.414027
          b         d         a
0  1.331971 -0.072199  1.775042
1 -1.162573 -0.485114  0.670196


,a,b,c,d
0,-1.197042,-1.550646,0.122298,-0.214030
1,-1.133196,1.167596,-0.583023,-0.603532
2,0.882659,1.173026,-0.456927,-0.414027
3,1.775042,1.331971,NaN,-0.072199
4,0.670196,-1.162573,NaN,-0.485114


In [110]:
df.loc[1, :]

a   -1.133196
b    1.167596
c   -0.583023
d   -0.603532
Name: 1, dtype: float64

## 2.4 联合重叠数据

In [111]:
a = pd.Series([np.nan, 2.5, 0, 3.5, 4.5, np.nan], 
              index=list('fedcba'))
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.], 
              index=list('abcdef'))
print(a)
print(b)

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64
a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64


In [112]:
# Series下的combine_first方法
b.combine_first(a)  # 返回一个新对象

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [113]:
# 等价于使用pandas常见数据对齐逻辑的轴向操作
b[b.isnull()] = a
print(b)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64


In [114]:
df1 = pd.DataFrame({'a': [1, np.nan, 5, np.nan], 
                    'b': [np.nan, 2, np.nan, 6], 
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7], 
                    'b': [np.nan, 3, 4, 6, 8]})
# DataFrame中的combine_first方法
# 逐列做相同的操作，根据传入的对象来“修补”调用对象的缺失值
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 3. 重塑和透视
## 3.1 使用多层索引进行重塑
- stack
- unstack

In [115]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorade'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorade,3,4,5


In [116]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorade  one       3
          two       4
          three     5
dtype: int64

In [117]:
result.unstack()  # 默认情况下，最内层是已拆堆的

number,one,two,three
state,,,
Ohio,0,1,2
Colorade,3,4,5


In [118]:
# 传入一个层级序号或名称来拆分一个不同的层级
# result.unstack(0)
result.unstack(level='state')

state,Ohio,Colorade
number,,
one,0,3
two,1,4
three,2,5


In [119]:
s1 = pd.Series([0, 1, 2, 3], index=list('abcd'))
s2 = pd.Series([4, 5, 6], index=list('cde'))
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [120]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [121]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [122]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [123]:
# 被拆堆的层级会变为结果中最低的层级
df = pd.DataFrame({'left': result, 'right': result+5}, 
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorade one        3      8
         two        4      9
         three      5     10

In [124]:
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorade    3   4     5     8   9    10

In [125]:
df.unstack('state')

side   left          right         
state  Ohio Colorade  Ohio Colorade
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [126]:
df.unstack('state').stack('side')

state         Colorade  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7